# Spark High-Level API

![img](https://static.packt-cdn.com/products/9781785888359/graphics/8bffbd94-04f7-46e3-a1e9-0d6046d2dcab.png)

Source: https://static.packt-cdn.com/products/9781785888359/graphics/8bffbd94-04f7-46e3-a1e9-0d6046d2dcab.png

## Overview of Spark SQL

Creating DataFrames
- [ ] RDD
    - [ ] createDataFrame()
- [ ] Text file
    - [ ] read.text()
- [ ] JSON file
    - [ ] read.json()
    - [ ] read,json(RDD)
- [ ] Parquet file
    - [ ] read.parquet()
- [ ] Table in a relational database
- [ ] Temporary table in Spark

DataFrame to RDD
- [ ] rdd()

Schemas
- [ ] Inferring schemas
    - [ ] Why it is not optimal practice
- [ ] Specifying schemas
    - [ ] Using StructType and StructField
    - [ ] Using DDL string (schema = “author STRING, title STRING, pages INT”)
- [ ] Metadata
    - [ ] printSchema()
    - [ ] columns()
    - [ ] dtypes()
- [ ] Actions
    - [ ] show()
- [ ] Transforms
    - [ ] select() and alias()
    - [ ] drop()
    - [ ] filter() / where()
    - [ ] distinct()
    - [ ] dropDuplicates()
    - [ ] sample
    - [ ] sampleBy()
    - [ ] limit()
    - [ ] orderBy() / sort()
    - [ ] groupBy()

Operations that return an RDD 
    - [ ] rdd.map()
    - [ ] rdd.flatMap()

pyspark.sql.functions module
- [ ] String functions
- [ ] Math functions
- [ ] Statistics functions
- [ ] Date functions
- [ ] Hashing functions
- [ ] Algorithms (sounded, levenstein)
- [ ] Windowing functions

User defined functions
- [ ] udf()
- [ ] pandas_udf()

Multiple DataFrames
- [ ] join(other, on, how)
- [ ] union(), unionAll()
- [ ] intersect()
- [ ] subtract()

Persistence
- [ ] cache()
- [ ] persist(:
- [ ] unpersist()
- [ ] cacheTable()
- [ ] clearCache()
- [ ] repartition()
- [ ] coalesce()

Output
- [ ] write.csv()
- [ ] write.parquet()
- [ ] write.json()

Spark SQL
- [ ] df.createOrReplaceTempView
- [ ] sql()
- [ ] table()

## Implementation notes

Spark 2.4 is desinged to run on Java 8 (or version 1.8). If you have veriosns of Java greater than Java 8, also install Java 8 and include this export

```bash
export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)
```

If using the latest `pyarrow` (`pip install -U pyarrow`), you will need to set this variable

```bash
ARROW_PRE_0_15_IPC_FORMAT=1
```

in the file

```bash
SPARK_HOME/conf/spark-env.sh
```


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
spark = (
    SparkSession.builder 
    .master("local") 
    .appName("BIOS-823") 
    .config("spark.executor.cores", 4) 
    .getOrCreate()    
)

In [ ]:
spark.version

In [ ]:
spark.conf.get('spark.executor.cores')

## Create a Spark DataFrame

In [ ]:
df = spark.range(3)

In [ ]:
df.show(3)

In [ ]:
%%file data/test.csv
number,letter
0,a
1,c
2,b
3,a
4,b
5,c
6,a
7,a
8,a
9,b
10,b
11,c
12,c
13,b
14,b

#### Implicit schema

In [ ]:
df = (
    spark.read.
    format('csv').
    option('header', 'true').
    option('inferSchema', 'true').
    load('data/test.csv')
)

In [ ]:
df.show(3)

In [ ]:
df.printSchema()

#### Explicit schema

For production use, you should provide an explicit schema to reduce risk of error.

In [ ]:
schema = T.StructType([
    T.StructField("number", T.DoubleType()),
    T.StructField("letter", T.StringType()),
])

In [ ]:
df = (
    spark.read.
    format('csv').
    option('header', 'true').
    schema(schema).
    load('data/test.csv')
)

In [ ]:
df.show(3)

In [ ]:
df.printSchema()

#### Alternative way to specify schema

You can use SQL DDL syntax to specify a schema as well.

In [ ]:
schema = 'number DOUBLE, letter STRING'

In [ ]:
df_altschema = (
    spark.read.
    format('csv').
    option('header', 'true').
    schema(schema=schema).
    load('data/test.csv')
)

In [ ]:
df_altschema.take(3)

In [ ]:
df_altschema.printSchema()

### Persist

In [ ]:
df.cache()

## Data manipulation

### Select

In [ ]:
df.select('number').show(3)

In [ ]:
from pyspark.sql.functions import col, expr

In [ ]:
df.select(col('number').alias('index')).show(3)

In [ ]:
df.select(expr('number as x')).show(3)

In [ ]:
df.withColumnRenamed('number', 'x').show(3)

## Filter

In [ ]:
df.filter('number % 2 == 0').show(3)

## Sort

In [ ]:
df.sort(df.number.desc()).show(3)

In [ ]:
df.orderBy(df.letter.desc()).show(3)

## Transform

In [ ]:
df.selectExpr('number*2 as x').show(3)

In [ ]:
df.withColumn('x', expr('number*2')).show(3)

## Sumarize

In [ ]:
import pyspark.sql.functions as F

In [ ]:
df.agg(F.min('number'),
       F.max('number'), 
       F.min('letter'), 
       F.max('letter')).show(3)

## Group by

In [ ]:
(
    df.groupby('letter').
    agg(F.mean('number'), F.stddev_samp('number')).show()
)

## Window functions

In [ ]:
from pyspark.sql.window import Window

In [ ]:
ws = (
    Window.partitionBy('letter').
    orderBy(F.desc('number')).
    rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

In [ ]:
df.groupby('letter').agg(F.sum('number')).show()

In [ ]:
df.show()

In [ ]:
(
    df.select('letter', F.sum('number').
              over(ws).
              alias('rank')).show()
)

## SQL

In [ ]:
df.createOrReplaceTempView('df_table')

In [ ]:
spark.sql('''SELECT * FROM df_table''').show(3)

In [ ]:
spark.sql('''
SELECT letter, mean(number) AS mean, 
stddev_samp(number) AS sd from df_table
WHERE number % 2 = 0
GROUP BY letter
ORDER BY letter DESC
''').show()

## String operatons

In [ ]:
from pyspark.sql.functions import split, lower, explode

In [ ]:
import pandas as pd

In [ ]:
s = spark.createDataFrame(
    pd.DataFrame(
        dict(sents=('Thing 1 and Thing 2',
                    'The Quick Brown Fox'))))

In [ ]:
s.show()

In [ ]:
from pyspark.sql.functions import regexp_replace

In [ ]:
s1 = (
    s.select(explode(split(lower(expr('sents')), ' '))).
    sort('col')
)

In [ ]:
s1.show()

In [ ]:
s1.groupBy('col').count().show()

In [ ]:
s.createOrReplaceTempView('s_table')

In [ ]:
spark.sql('''
SELECT regexp_replace(sents, 'T.*?g', 'FOO')
FROM s_table
''').show()

### Numeric operations

In [ ]:
from pyspark.sql.functions import log1p, randn

In [ ]:
df.selectExpr('number', 'log1p(number)', 'letter').show(3)

In [ ]:
(
    df.selectExpr('number', 'randn() as random').
    stat.corr('number', 'random')
)

### Date and time

In [ ]:
dt = (
    spark.range(3).
    withColumn('today', F.current_date()).
    withColumn('tomorrow', F.date_add('today', 1)).
    withColumn('time', F.current_timestamp())
)

In [ ]:
dt.show()

In [ ]:
dt.printSchema()

### Nulls

In [ ]:
%%file data/test_null.csv
number,letter
0,a
1,
2,b
3,a
4,b
5,
6,a
7,a
8,
9,b
10,b
11,c
12,
13,b
14,b

In [ ]:
dn = (
    spark.read.
    format('csv').
    option('header', 'true').
    option('inferSchema', 'true').
    load('data/test_null.csv')
)

In [ ]:
dn.printSchema()

In [ ]:
dn.show()

In [ ]:
dn.na.drop().show()

In [ ]:
dn.na.fill('Missing').show()

## UDF

To avoid degrading performance, avoid using UDF if you can use the functions in `pyspark.sql.functions`. If you must use UDFs, prefer `pandas_udf` to `udf` where possible.

In [ ]:
from pyspark.sql.functions import udf, pandas_udf

### Standard Python UDF

In [ ]:
@udf('double')
def square(x):
    return x**2

In [ ]:
df.select('number', square('number')).show(3)

### Pandas UDF

In [ ]:
@pandas_udf('double')
def scale(x):
    return (x - x.mean())/x.std()

In [ ]:
df.select(scale('number')).show(3)

#### Grouped agg

In [ ]:
@pandas_udf('double', F.PandasUDFType.GROUPED_AGG)
def gmean(x):
    return x.mean()

In [ ]:
df.groupby('letter').agg(gmean('number')).show()

#### Grouped map

In [ ]:
@pandas_udf(df.schema, F.PandasUDFType.GROUPED_MAP)
def gscale(pdf):
    return pdf.assign(
        number = (pdf.number - pdf.number.mean()) /
        pdf.number.std())

In [ ]:
df.groupby('letter').apply(gscale).show()

## Joins

In [ ]:
names = 'ann ann bob bob chcuk'.split()
courses = '821 823 821 824 823'.split()
pdf1 = pd.DataFrame(dict(name=names, course=courses))

In [ ]:
pdf1

In [ ]:
course_id = '821 822 823 824 825'.split()
course_names = 'Unix Python R Spark GLM'.split()
pdf2 = pd.DataFrame(dict(course_id=course_id, name=course_names))

In [ ]:
pdf2

In [ ]:
df1 = spark.createDataFrame(pdf1)
df2 = spark.createDataFrame(pdf2)

In [ ]:
df1.join(df2, df1.course == df2.course_id, how='inner').show()

In [ ]:
df1.join(df2, df1.course == df2.course_id, how='right').show()

## DataFrame conversions

In [ ]:
sc = spark.sparkContext

In [ ]:
rdd = sc.parallelize([('ann', 23), ('bob', 34)])

In [ ]:
df = spark.createDataFrame(rdd, schema='name STRING, age INT')

In [ ]:
df.show()

In [ ]:
df.rdd.map(lambda x: (x[0], x[1]**2)).collect()

In [ ]:
df.rdd.mapValues(lambda x: x**2).collect()

In [ ]:
df.toPandas()